In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")



In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(
    model_name="openai/gpt-oss-20b",
    api_key=groq_api_key,
    temperature=0.5,
    max_tokens=8192,
    top_p=0.95,
)
model

d:\Python\Python-Programming\Complete_Agentic_AI_Bootcamp_with_LangGraph_and_LangChain\Section_8_Building_AI_Agents\venv\lib\site-packages\pydantic\main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000190FE5B7E80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000190FE5B6FE0>, model_name='openai/gpt-oss-20b', temperature=0.5, model_kwargs={'top_p': 0.95}, groq_api_key=SecretStr('**********'), max_tokens=8192)

In [7]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model.invoke([HumanMessage(content="Hello, how are you?")])

AIMessage(content='Hello! I’m doing great—thanks for asking. How about you? Anything interesting on your mind today?', additional_kwargs={'reasoning_content': 'We need to respond as ChatGPT. The user says "Hello, how are you?" It\'s a greeting. We should respond politely, perhaps ask how they are. No constraints.'}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 77, 'total_tokens': 145, 'completion_time': 0.066662205, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.003688563, 'prompt_tokens_details': None, 'queue_time': 0.197960157, 'total_time': 0.070350768}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e23fc997ca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--cdfcd242-c874-4f10-a9aa-e0ddb1b4d540-0', usage_metadata={'input_tokens': 77, 'output_tokens': 68, 'total_tokens': 145, 'output_token_details': {'reasoning': 37}})

In [8]:
model.invoke([
    HumanMessage(content="Hello, My name is John Doe.I am a software engineer.I am 30 years old.I live in San Francisco.I am a good person."),
    AIMessage(content="Hello, John Doe. How can I help you today?"),
    HumanMessage(content="Hey What is my name and age? Where do I live?")
    ])

AIMessage(content='You mentioned that your name is **John\u202fDoe**, you’re **30\u202fyears old**, and you live in **San\u202fFrancisco**.', additional_kwargs={'reasoning_content': "User wants to know their name, age, and location. According to policy, we can provide that because it's user-provided data. It's not sensitive. So we can answer."}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 135, 'total_tokens': 211, 'completion_time': 0.096444218, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.223708177, 'prompt_tokens_details': None, 'queue_time': 0.362933045, 'total_time': 0.320152395}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_24bfb4a850', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--25d70e2d-0fc9-4fd0-a938-966046e305af-0', usage_metadata={'input_tokens': 135, 'output_tokens': 76, 'total_tokens': 211, 'output_token_details': {'reasoning': 37}}

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}
with_message_history.invoke(
    HumanMessage(content="Hello, My name is John Doe.I am a software engineer.I am 30 years old.I live in San Francisco.I am a good person."),
    config=config
)




AIMessage(content='Hi John! 👋 It’s great to meet a fellow software engineer from San\u202fFrancisco. How can I help you today? Whether you’re looking for coding tips, project ideas, or just want to chat about the latest tech trends, I’m here for you!', additional_kwargs={'reasoning_content': 'The user says: "Hello, My name is John Doe. I am a software engineer. I am 30 years old. I live in San Francisco. I am a good person." They are basically introducing themselves. We need to respond. There\'s no explicit request. We can respond with a friendly greeting, maybe ask follow-up. We should maintain conversation. Also we might reflect back some of the info. No conflict. Just respond.'}, response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 100, 'total_tokens': 253, 'completion_time': 0.160138931, 'completion_tokens_details': {'reasoning_tokens': 89}, 'prompt_time': 0.006696352, 'prompt_tokens_details': None, 'queue_time': 0.047710078, 'total_time': 0.166835283}, 'mo

In [13]:
with_message_history.invoke(
    HumanMessage(content="Hey What is my name and age? Where do I live?"),
    config=config
)


AIMessage(content='You’re John\u202fDoe, 30\u202fyears old, and you live in San\u202fFrancisco.', additional_kwargs={'reasoning_content': 'We need to respond with the information the user provided: name John Doe, age 30, live in San Francisco. Probably just repeat that.'}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 178, 'total_tokens': 238, 'completion_time': 0.061163478, 'completion_tokens_details': {'reasoning_tokens': 30}, 'prompt_time': 0.014832029, 'prompt_tokens_details': None, 'queue_time': 0.051878731, 'total_time': 0.075995507}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--ec0e3fb7-3bcf-4d21-bc41-da70dceb509f-0', usage_metadata={'input_tokens': 178, 'output_tokens': 60, 'total_tokens': 238, 'output_token_details': {'reasoning': 30}})

In [14]:
## Changing session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    HumanMessage(content="What is my name and age?"),
    config=config1
)

response





AIMessage(content='I’m sorry, but I don’t have any information about your name or age. If you’d like to share that, I can keep it in mind for our conversation!', additional_kwargs={'reasoning_content': 'User asks: "What is my name and age?" We don\'t have that info. According to policy, we must not guess or fabricate. We should ask for clarification or say we don\'t know. So we should respond politely that we don\'t have that information.'}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 78, 'total_tokens': 174, 'completion_time': 0.10049759, 'completion_tokens_details': {'reasoning_tokens': 52}, 'prompt_time': 0.005111536, 'prompt_tokens_details': None, 'queue_time': 0.053361864, 'total_time': 0.105609126}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8948b448-6905-498c-a05b-1a32afaa1ec2-0', usage_metadata={'input_to

In [16]:
response = with_message_history.invoke(
    HumanMessage(content="Hey My name is John Doe.I am 30 years old.I live in San Francisco.I am a good person."),
    config=config1
)
response

AIMessage(content='Got it—John, 30\u202fyears old, San\u202fFrancisco, and a good person! How can I help you today?', additional_kwargs={'reasoning_content': 'User repeats same statement. Likely wants confirmation or maybe wants something else. We should respond acknowledging. Possibly ask what they want.'}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 211, 'total_tokens': 275, 'completion_time': 0.064407667, 'completion_tokens_details': {'reasoning_tokens': 27}, 'prompt_time': 0.01437326, 'prompt_tokens_details': None, 'queue_time': 0.017987668, 'total_time': 0.078780927}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d23c14756c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--a17a73a4-d48e-4aab-95be-a69ace05e0c8-0', usage_metadata={'input_tokens': 211, 'output_tokens': 64, 'total_tokens': 275, 'output_token_details': {'reasoning': 27}})

In [18]:
response = with_message_history.invoke(
    HumanMessage(content="What is my name and age?"),
    config=config1
)
response

AIMessage(content='Your name is **John\u202fDoe** and you’re **30\u202fyears old**.', additional_kwargs={'reasoning_content': 'The user repeats question. We have to answer with name and age. We should not mention policy. Just answer.'}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 291, 'total_tokens': 342, 'completion_time': 0.049989025, 'completion_tokens_details': {'reasoning_tokens': 24}, 'prompt_time': 0.014088099, 'prompt_tokens_details': None, 'queue_time': 0.019329709, 'total_time': 0.064077124}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_5979a0e1b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--4206971a-3e75-47a7-9534-e5992dcc6730-0', usage_metadata={'input_tokens': 291, 'output_tokens': 51, 'total_tokens': 342, 'output_token_details': {'reasoning': 24}})